# AIRMASS

## Overview
The `AIRMASS` function calculates the mass of air per square meter in the atmosphere along a given path, based on the provided atmospheric model. This is useful for applications in astronomy, meteorology, and solar energy, where the amount of air mass affects the transmission of light and radiation through the atmosphere. The calculation integrates the density of the atmosphere along the path from the observer to the top of the atmosphere, accounting for the angle above the horizon.

The air mass $m(\gamma)$ is given by:

```math
m(\gamma) = \int_0^\infty \rho \left\{1 - \left[1 + 2(RI-1)(1-\rho/\rho_0)\right][\cos\gamma(1+h/R)]^2\right\}^{-1/2} dH
```

where:
- $\rho$ is the density of the atmosphere as a function of elevation $H$
- $\gamma$ is the angle above the horizon (in degrees)
- $R$ is the radius of the planet
- $RI$ is the refractive index of the atmosphere
- $\rho_0$ is the density at sea level

## Usage
To use this function in Excel:

```excel
=AIRMASS(angle, [H_max], [R_planet], [RI])
```

- `angle` is required; other arguments are optional.

## Arguments
| Argument   | Type  | Required | Description                                                      | Example   |
|:-----------|:------|:---------|:-----------------------------------------------------------------|:----------|
| angle      | float | Required | Degrees above the horizon (90 = straight up)                     | 90        |
| H_max      | float | Optional | Maximum height to integrate to [m]                               | 86400     |
| R_planet   | float | Optional | Radius of the planet [m]                                         | 6371229   |
| RI         | float | Optional | Refractive index of the atmosphere [-]                           | 1.000276  |

## Returns
| Returns | Type  | Description                                 | Example   |
|:--------|:------|:--------------------------------------------|:----------|
| m       | float | Mass of air per square meter [kg/m^2]        | 10356.12  |
| Error   | str   | Error message if calculation fails            | "Error: Invalid input" |

## Examples
### Calculating Air Mass at Zenith
| angle |
|:-----|
| 90    |

```excel
=AIRMASS(90)
```
Expected output: 10356.12

### Calculating Air Mass at 45 Degrees with Custom Parameters
| angle | H_max | R_planet | RI      |
|:-----|:------|:---------|:--------|
| 45    | 90000 | 6371000  | 1.0003  |

```excel
=AIRMASS(45, 90000, 6371000, 1.0003)
```
Expected output: (varies depending on model)

## Reference
See the `fluids` package [GitHub repository](https://github.com/CalebBell/fluids) and [documentation](https://fluids.readthedocs.io/index.html) for details.  We are grateful to the package authors for their work.

In [ ]:
import micropip
await micropip.install('fluids')
from fluids.atmosphere import airmass, ATMOSPHERE_1976

def airmass(angle, H_max=86400.0, R_planet=6371229.0, RI=1.000276):
    """
    Calculates the mass of air per square meter in the atmosphere along a given path.

    Args:
        angle (float): Degrees above the horizon (90 = straight up).
        H_max (float, optional): Maximum height to integrate to [m]. Default is 86400.
        R_planet (float, optional): Radius of the planet [m]. Default is 6371229.
        RI (float, optional): Refractive index of the atmosphere [-]. Default is 1.000276.

    Returns:
        float: Mass of air per square meter [kg/m^2], or error message as str.
    """
    try:
        result = airmass(lambda Z: ATMOSPHERE_1976(Z).rho, angle, H_max, R_planet, RI)
        return result
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
%pip install -q ipytest
import ipytest
ipytest.autoconfig()

def test_airmass_zenith():
    result = airmass(90)
    assert isinstance(result, float)
    assert result > 9000 and result < 12000

def test_airmass_45deg_custom():
    result = airmass(45, 90000, 6371000, 1.0003)
    assert isinstance(result, float) or isinstance(result, str)
    if isinstance(result, float):
        assert result > 0

def test_airmass_invalid():
    result = airmass('bad_input')
    assert isinstance(result, str)
    assert "Error" in result

ipytest.run('-s')

In [ ]:
import gradio as gr

examples = [
    [90, 86400.0, 6371229.0, 1.000276],
    [45, 90000, 6371000, 1.0003],
]

demo = gr.Interface(
    fn=airmass,
    inputs=[
        gr.Number(label='angle', value=90),
        gr.Number(label='H_max', value=86400.0),
        gr.Number(label='R_planet', value=6371229.0),
        gr.Number(label='RI', value=1.000276),
    ],
    outputs=gr.Number(label='m'),
    examples=examples,
    description='Calculates the mass of air per square meter in the atmosphere along a given path, based on the provided atmospheric model. Useful for astronomy, meteorology, and solar energy applications.',
    flagging_mode='never'
)
demo.launch()